In [14]:
import fiftyone.zoo as foz

import fiftyone as fo

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import torch
import fiftyone.utils.coco as fouc
from PIL import Image

from torch.utils.data import DataLoader as DataLoader


In [2]:
dataset = foz.load_zoo_dataset("coco-2017", split="validation")

 100% |██████|    1.9Gb/1.9Gb [1.6m elapsed, 0s remaining, 20.2Mb/s]      
Extracting annotations to 'C:\Users\apole\fiftyone\coco-2017\raw\instances_val2017.json'
 100% |██████|    6.1Gb/6.1Gb [8.4m elapsed, 0s remaining, 13.9Mb/s]      
Extracting images to 'C:\Users\apole\fiftyone\coco-2017\validation\data'
Writing annotations to 'C:\Users\apole\fiftyone\coco-2017\validation\labels.json'
Dataset info written to 'C:\Users\apole\fiftyone\coco-2017\info.json'
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [1.1m elapsed, 0s remaining, 96.5 samples/s]       
Dataset 'coco-2017-validation' created


In [3]:
dataset.compute_metadata()

Loading your custom data

If you have data that follows a certain format on disk (for example a directory tree for classification, the COCO detection format, or many more), then you can load it into FiftyOne in one line of code:


dataset = fo.Dataset(
    "/path/to/dataset_dir", 
    dataset_type=fo.types.COCODetectionDataset, 
    name="my_dataset"
)


In [8]:
def get_model(num_classes):
    # load a model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

In [10]:
class FiftyOneTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a FiftyOne dataset.
    
    Args:
        fiftyone_dataset: a FiftyOne dataset or view that will be used for 
            training or testing
        transforms (None): a list of PyTorch transforms to apply to images 
            and targets when loading
        gt_field ("ground_truth"): the name of the field in fiftyone_dataset 
            that contains the desired labels to load
        classes (None): a list of class strings that are used to define the 
            mapping between class names and indices. If None, it will use 
            all classes present in the given fiftyone_dataset.
    """

    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="ground_truth",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        img = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []
        area = []
        iscrowd = []
        detections = sample[self.gt_field].detections
        for det in detections:
            coco_obj = fouc.COCOObject.from_detection(
                det, metadata, labels_map_rev=self.labels_map_rev
            )
            x, y, w, h = coco_obj.bbox
            boxes.append([x, y, x + w, y + h])
            labels.append(coco_obj.category_id)
            area.append(coco_obj.area)
            iscrowd.append(coco_obj.iscrowd)

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.as_tensor([idx])
        target["area"] = torch.as_tensor(area, dtype=torch.float32)
        target["iscrowd"] = torch.as_tensor(iscrowd, dtype=torch.int64)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.img_paths)

    def get_classes(self):
        return self.classes

In [12]:
model = get_model(80)

In [13]:
dataset = FiftyOneTorchDataset(dataset)

In [15]:
my_loader = DataLoader(dataset, batch_size=5, shuffle=False, num_workers=2)

In [17]:
# for x in my_loader:
#     print(x)